# [IAPR 2020:][iapr2020] Lab 1 ‒  Image segmentation

**Authors:** Claire Meyer, Nicolas Furrer, Philipp Schuler  
**Due date:** 26.03.2020

[iapr2018]: https://github.com/LTS5/iapr-2020

## Extract relevant data
We first need to extract the `lab-01-data.tar.gz` archive.
To this end, we use the [tarfile] module from the Python standard library.

[tarfile]: https://docs.python.org/3.6/library/tarfile.html

In [ ]:
import tarfile
import os

data_base_path = os.path.join(os.pardir, "data")
data_folder = "lab-01-data"
#tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
#with tarfile.open(tar_path, mode='r:gz') as tar:
#    tar.extractall(path=data_base_path)

## Part 1: Brain segmentation

Your goal: compute the size of the brain (in pixels) in a 2D image of a human head taken by Magnetic Resonance Imaging (MRI).
* Try as many methods as you can, the more the better.
* At least region growing and contour detection.

### 1.1 Brain image visualization

In [ ]:
import skimage.io
import matplotlib.pyplot as plt
%matplotlib inline

import cv2 as cv
import numpy as np

# Load image
data_path = os.path.join(data_base_path, data_folder)
brain_im = skimage.io.imread(os.path.join(data_path, "brain-slice40.tiff"))
im_h, im_w = brain_im.shape

fig, ax = plt.subplots(1,1,figsize=(6, 6))
ax.imshow(brain_im, cmap="gray")
ax.set_title("MRI brain image ({} px, {} px)".format(im_h, im_w))
ax.axis("off")
plt.show()
print("Done")

### 1.2 Region growing
Add your implementation and discussion

In [ ]:
def region_growing_openCV(image, seed, low_threshold, high_threshold):
    mask = np.zeros((im_h+2,im_w+2),np.uint8)
    cv.floodFill(image, mask, seed, 255,low_threshold, high_threshold)
    return image, mask
    


brain_im_c = brain_im.copy()
seed = (150,150)
image, mask = region_growing_openCV(brain_im_c, seed, 6, 6)
mask_h, mask_w = mask.shape
count = 0
for x in range(mask_w):
    for y in range(mask_h):
        if mask[x,y]==1:
            count = count + 1


#Plot
print("Seed : " , seed)
print("Brain size : ", count, " Pixels")
ax = plt.subplot(1,3,1)
ax.set_title("Base image")
ax.imshow(brain_im, cmap='gray')
ax.axis('off')
ax = plt.subplot(1,3,2)
ax.set_title("Brain detection")
ax.imshow(image, cmap='gray')
ax.axis('off')
ax = plt.subplot(1,3,3,)
ax.set_title("Brain mask")
ax.imshow(mask, cmap ='gray')
ax.axis('off')


#### Discussion
Cest la la discussion

### 1.3 Contour detection
Add your implementation and discussion

In [ ]:
import numpy as np
import cv2 as cv

# Image in open CV format
original_img = cv.imread(os.path.join(data_path, "brain-slice40.tiff"),0)

# # Histogram equalization 
# equalized_img = cv.equalizeHist(original_img)

# Canny detection 
edges_img = cv.Canny(image=original_img,threshold1=85,threshold2=200,apertureSize=3)

# Dilation of contours
kernel = np.zeros((3,3),np.uint8)
cv.circle(img=kernel, center=(1,1), radius=1, color=255, thickness=-1) 
dilated_edges_img = cv.dilate(edges_img,kernel,iterations = 1)

# # Canny detection 
# t1 = 240
# edges_img_2 = cv.Canny(image=equalized_img,threshold1=t1,threshold2=t1+5,apertureSize=3)

# # Dilation of contours
# dilated_edges_img_2 = cv.dilate(edges_img_2,kernel,iterations = 1)

# Flood-filling from pixel (150, 150)
h, w = edges_img.shape[:2]
mask = np.zeros((h+2, w+2), np.uint8)
flooded_img = dilated_edges_img.copy()
cv.floodFill(flooded_img, mask, (150,150), 255);

# Inversion of the flood filled image 
inverted_flooded_img = cv.bitwise_not(flooded_img)

# Combination of the thresholded image with the inverted flood filled image using bitwise OR operation 
dilated_segmented_img = cv.bitwise_or(dilated_edges_img,inverted_flooded_img)
segmented_img = cv.erode(dilated_segmented_img,kernel,iterations = 1)

# Brain
brain = cv.bitwise_not(segmented_img)
superimposed_img = cv.add(src1=brain,src2=original_img)



######################
# ALREADY IN MASK
######################


# # Closing of contours
# closed_edges_img = cv.morphologyEx(src=edges_img,op=cv.MORPH_CLOSE,kernel=kernel,iterations = 1)

# # Flood-filling from pixel (150, 150)
# mask_2 = np.zeros((h+2, w+2), np.uint8)
# flooded_img_2 = closed_edges_img.copy()
# cv.floodFill(flooded_img_2, mask_2, (150,150), 255);

# # Inversion of the flood filled image 
# inverted_flooded_img_2 = cv.bitwise_not(flooded_img_2)

# # Combination of the thresholded image with the inverted flood filled image using bitwise OR operation 
# segmented_img_2 = cv.bitwise_or(closed_edges_img,inverted_flooded_img_2)

# Number of pixels
n = np.sum(segmented_img == 255)
print(f"number of white pixels : {n}")
print(f"proportion with respect to image size : {n/(256*256)}")

# Save 
# cv.imwrite(filename="test.png", img=flooded_img_2)

# Plots
N = 3
M = 3
fig, ax = plt.subplots(N,M,figsize=(M*6,N*6))

ax[0,0].imshow(original_img,cmap = 'gray')
ax[0,0].set_title("Original image")
ax[0,1].imshow(edges_img,cmap = 'gray')
ax[0,2].imshow(dilated_edges_img,cmap = 'gray')

# ax[1,0].imshow(closed_edges_img,cmap = 'gray')
# ax[1,1].imshow(mask_2,cmap = 'gray')
# ax[1,2].imshow(cv.bitwise_not(segmented_img_2),cmap = 'gray')

ax[1,0].imshow(flooded_img,cmap = 'gray')
ax[1,1].imshow(inverted_flooded_img,cmap = 'gray')
ax[1,2].imshow(dilated_segmented_img,cmap = 'gray')

ax[2,0].imshow(segmented_img,cmap = 'gray')
ax[2,1].imshow(brain,cmap = 'gray')
ax[2,2].imshow(superimposed_img,cmap = 'gray')

for a in ax.flatten():
    a.axis("off")


### 1.4 Additional method(s)
Add your implementation and discussion

## Part 2: Shape/color segmentation

You will find hereafter three pictures taken under three different illuminations, containing some shapes with different colors. We ask you to create a routine to:

1. Count the number of shapes of each color.
2. Compute the total area (in pixels) of each color.

Please note that one specific challenge is to be robust to illumination changes. Therefore some kind of intensity normalization should probably be used.

**Note:** the routine(s) that you will write for this exercise will be useful for the final project as well, so pay special attention to it.

### 2.1 Visualization

In [ ]:
# Load images
im_names = ['arena-shapes-01', 'arena-shapes-02', 'arena-shapes-03']
filenames = [os.path.join(data_path, name) + '.png' for name in im_names]
ic = skimage.io.imread_collection(filenames)
images = skimage.io.concatenate_images(ic)
print('Number of images: ', images.shape[0])
print('Image size: {}, {} '.format(images.shape[1], images.shape[2]))
print('Number of color channels: ', images.shape[-1])

In [ ]:
# Plot images
fig, axes = plt.subplots(1, 3, figsize=(12, 12))
for ax, im, nm in zip(axes.ravel(), images, im_names):
    ax.imshow(im)
    ax.axis('off')
    ax.set_title(nm)
plt.show()

### 2.2 Number of shapes of each color
Add your implementation and discussion

### 2.3 Total area (in pixels) of each color
Add your implementation and discussion

In [ ]:
print(f"area 1 = {stats[0][cv.CC_STAT_AREA]}")
print(f"area 2 = {stats[1][cv.CC_STAT_AREA]}")
print(f"proportion = {stats[0][cv.CC_STAT_AREA]/stats[1][cv.CC_STAT_AREA]}")

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)

dt = 0.01
t = np.arange(0, 30, dt)
nse1 = np.random.randn(len(t))                 # white noise 1
nse2 = np.random.randn(len(t))                 # white noise 2

# Two signals with a coherent part at 10Hz and a random part
s1 = np.sin(2 * np.pi * 10 * t) + nse1
s2 = np.sin(2 * np.pi * 10 * t) + nse2

fig, axs = plt.subplots(2, 1)
axs[0].plot(t, s1, t, s2)
axs[0].set_xlim(0, 2)
axs[0].set_xlabel('time')
axs[0].set_ylabel('s1 and s2')
axs[0].grid(True)

cxy, f = axs[1].cohere(s1, s2, 256, 1. / dt)
axs[1].set_ylabel('coherence')

fig.tight_layout()
plt.show()
